# Zoom data reduction

## Introduction

This notebook is an example of how ESSsans can be used to reduce data from [Zoom at ISIS](https://www.isis.stfc.ac.uk/Pages/Zoom.aspx).
The following description is kept relatively brief, for more context see the rest of the documentation.
In particular the [Sans2d](./sans2d.ipynb) notebook may be useful.

There are a few things that are not yet handled:

- Calibration file is not loaded or handled yet.
- TOF or wavelength masks
- log binning
- position corrections from use file

## Setup

### Imports and configuration

In [ ]:
from mantid import ConfigService

cfg = ConfigService.Instance()
cfg.setLogLevel(3)  # Silence verbose load via Mantid

In [ ]:
import scipp as sc
import sciline
import scippneutron as scn
import plopp
import esssans as sans
import esssans.isis
from esssans.types import *

### Setup input files

In [ ]:
root = 'zoom_data'
params = {
    sans.types.DirectBeamFilename: f'{root}/Direct_Zoom_4m_8mm_100522.txt',
    sans.isis.CalibrationFilename: f'{root}/192tubeCalibration_11-02-2019_r5_10lines.nxs',
    sans.isis.Filename[sans.types.SampleRun]: f'{root}/ZOOM00034786.nxs',
    sans.isis.Filename[sans.types.EmptyBeamRun]: f'{root}/ZOOM00034787.nxs',
}
masks = [
    'andru_test.xml',
    'left_beg_18_2.xml',
    'right_beg_18_2.xml',
    'small_bs_232.xml',
    'small_BS_31032023.xml',
    'tube_1120_bottom.xml',
    'tubes_beg_18_2.xml',
]
mask_paths = [f'{root}/{mask}' for mask in masks]
masks = sciline.ParamTable(str, {sans.isis.PixelMaskFilename: mask_paths}, index=masks)

### Setup reduction parameters

In [ ]:
params[NeXusMonitorName[Incident]] = 'monitor3'
params[NeXusMonitorName[Transmission]] = 'monitor5'

band = sc.linspace('wavelength', 1.75, 16.5, num=2, unit='angstrom')
params[WavelengthBands] = band
params[WavelengthBins] = sc.geomspace(
    'wavelength', start=band[0], stop=band[-1], num=141
)

params[QBins] = sc.geomspace(dim='Q', start=0.004, stop=0.8, num=141, unit='1/angstrom')
params[NonBackgroundWavelengthRange] = sc.array(
    dims=['wavelength'], values=[0.7, 17.1], unit='angstrom'
)
params[CorrectForGravity] = True
params[UncertaintyBroadcastMode] = UncertaintyBroadcastMode.upper_bound

### Setup reduction pipeline

In [ ]:
providers = sans.providers + sans.isis.providers
providers = providers + (
    sans.isis.transmission_from_background_run,
    sans.isis.transmission_from_sample_run,
)
pipeline = sciline.Pipeline(providers, params=params)
pipeline.set_param_table(masks)

## Reduction

### The reduction workflow

In [ ]:
iofq = pipeline.get(IofQ[SampleRun])
iofq.visualize(graph_attr={'rankdir': 'LR'})

### Running the workflow

In [ ]:
da = iofq.compute()
da.plot(norm='log', scale={'Q': 'log'})

### Inspecting intermediate results

In [ ]:
monitors = (
    WavelengthMonitor[SampleRun, Incident],
    WavelengthMonitor[SampleRun, Transmission],
)
parts = (CleanSummedQ[SampleRun, Numerator], CleanSummedQ[SampleRun, Denominator])
iofqs = (IofQ[SampleRun],)
keys = monitors + (MaskedData[SampleRun],) + parts + iofqs

results = pipeline.compute(keys)

display(sc.plot({str(key): results[key] for key in monitors}, norm='log'))

display(
    scn.instrument_view(
        results[MaskedData[SampleRun]].hist(),
        pixel_size=0.0075,
        norm='log',
        camera=plopp.graphics.Camera(position=(0, 0, 22)),
    )
)

parts = {str(key): results[key] for key in parts}
parts = {key: val if val.bins is None else val.hist() for key, val in parts.items()}
display(sc.plot(parts, norm='log', scale={'Q': 'log'}))

iofqs = {str(key): results[key] for key in iofqs}
iofqs = {key: val if val.bins is None else val.hist() for key, val in iofqs.items()}
display(sc.plot(iofqs, norm='log', scale={'Q': 'log'}))